In [3]:
import sys
sys.path.append("../")

In [4]:
import numpy
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
from tqdm import tqdm
import os,re

In [5]:
def calc_density_with_mean(heatmap_benign,
                            heatmap_malignant,
                            bbox,
                        ):
    
    
    h, w = heatmap_benign.shape

    #this is used for rescaling the coordinates of boxes
    scale_factor = numpy.array([w, h, w, h], dtype=float)
    
    
    x_min, y_min, x_max, y_max = (bbox * scale_factor).astype(int)    
    
    dens_ben = heatmap_benign[y_min:y_max, x_min:x_max].mean()
    dens_mal = heatmap_malignant[y_min:y_max, x_min:x_max].mean()


    return dens_ben, dens_mal

In [6]:
def load_and_process_with_mean_multiple_only_benign(threshold_benign):
  
    if not isinstance(threshold_benign, numpy.ndarray):
        raise TypeError(f"expected threshold_benign to be numpy.ndarra, got {type(threshold_benign)}")

    if not (threshold_benign.ndim == 1):
        raise ValueError(f"expected threshold_benign to have shape [N], but got {threshold_benign.shape}")

    if not numpy.all((0 < threshold_benign) & (threshold_benign < 1)):
        raise ValueError(f"expected threshold_benign values to be in range 0 to 1, but got {threshold_benign}")

    
    path = "../Sample_data/generated_heatmaps"

    files = os.listdir(path)
    files.sort(key = lambda x: int(re.match(r"image(\d+).+\.npy", x)[1]))

    
    N = len(files) // 3
    M = threshold_benign.shape[0]
    main_matrix = numpy.zeros((M, 2, 2))
    
    for idx in tqdm(range(N)):
        
        temp = files[3* idx:3*idx + 3]
        temp.sort(key = lambda x: 0 if "bbox" in x else 1 if "ben" in x else 2)
        
        with open(os.path.join(path, temp[0]), 'rb') as f:

            bbox = numpy.load(f)

        with open(os.path.join(path, temp[1]), 'rb') as f:

            heatmap_benign = numpy.load(f)

        #the re.match used to get the annotations from the file names
        annot = re.match(".+_(.+).npy", temp[1])[1]
        

        dens_ben, dens_mal = calc_density_with_mean(heatmap_benign, heatmap_benign, bbox)


        if annot in ["2", "3"]:
            true_label = 1
        else:
            true_label = 0


        is_benign = dens_ben > threshold_benign

        main_matrix[is_benign, true_label, 1] += 1        
        main_matrix[~is_benign, true_label, 0] += 1

    specificity = numpy.zeros((M, 2))    
    sensitivity = numpy.zeros((M, 2))
    
    
   ##specificity
    
    specificity[:, 0] = (main_matrix[:, 1, 1])/\
    (main_matrix[:, 1, 0] + main_matrix[:, 1, 1])
    
    specificity[:, 1] = (main_matrix[:, 0, 0])/\
    (main_matrix[:, 0, 1] + main_matrix[:, 0, 0])

    
    ##sensitivity
    
    sensitivity[:, 0] = main_matrix[:, 0, 0]/(main_matrix[:, 0].sum(axis=-1))
    sensitivity[:, 1] = main_matrix[:, 1, 1]/(main_matrix[: ,1].sum(axis=-1))
    
    #mean accuracy
    
    corrects = main_matrix[:, 0, 0] + main_matrix[:, 1, 1]
    mean_accuracy = corrects / main_matrix.sum(axis=(-1, -2))
    
    return main_matrix, specificity, sensitivity, mean_accuracy

# Gridsearch over the threshold values

In [14]:
t_b = numpy.linspace(0.1, 0.99999, 1000)

main_matrix, specificity, sensitivity, mean_accuracy = \
load_and_process_with_mean_multiple_only_benign(t_b)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 316/316 [00:00<00:00, 1656.54it/s]


# Best mean accuracy value

In [16]:
best_index = numpy.argmax(mean_accuracy)
mean_accuracy[best_index]

0.7120253164556962

# Threshold corresponding the best mean accuracy index

In [17]:
t_b[best_index]

0.33963697697697703

# Sensitivity corresponding the best mean accuracy index

In [20]:
sensitivity[best_index]

array([0.86384977, 0.39805825])